In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import keras as ks
import matplotlib.pyplot as plt
from keras import layers
from keras import models
from keras import backend as k
from keras.applications.inception_v3 import InceptionV3
from keras.applications import InceptionResNetV2
from keras.models import Model, model_from_json
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout, Conv2D, GlobalAveragePooling2D
from keras import optimizers
from keras import regularizers



In [ ]:
base_model = InceptionV3(weights='imagenet', include_top = False, input_shape = (256, 256, 3))

for layer in base_model.layers[:-5]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense( 512, activation='relu')(x)  
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)  # new FC layer, random init
x = Dropout(0.2)(x)

predictions = Dense(4 , activation='softmax')(x)  # new softmax layer
model = Model(outputs=predictions, inputs=base_model.input)

adam = ks.optimizers.Adam(lr=1e-3, beta_1 = .9, beta_2 = .999)

model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train_path = ('../input/training/training')
test_path = ('../input/testing/testing')
# print(os.listdir("../input"))

# train_batches_gen = ImageDataGenerator( rescale=1./255,
#       rotation_range=10,
#       horizontal_flip=True,
#       width_shift_range=0.2, 
#       height_shift_range=0.2 )
train_batches_gen = ImageDataGenerator( rescale=1./255)
test_batches_gen = ImageDataGenerator( rescale=1./255)

train_batches = train_batches_gen.flow_from_directory(train_path, target_size = (256,256), batch_size = 128)
test_batches = test_batches_gen.flow_from_directory(test_path, target_size = (256,256))

In [ ]:
history = model.fit_generator(train_batches, epochs = 8, validation_data = test_batches)

